In [191]:
from sklearn.externals import joblib
import pandas as pd
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import re
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier


init_df = pd.read_csv('/Users/yiyangzhang/CS@Northeastern University/CS7180_AI/Wine-review-Project-master/wine-reviews_data/winemag-data-130k-v2.csv', na_values="?")
init_df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [192]:
## remove outliear
parsed_df = init_df.where(init_df['price'] < 1500)


## I drop these columns because some of them looks unrelated to final results(Unnamed: 0, taster_twitter_handle,designation), the others have 
## too many values(more than 1000 or 10000) so it is impossible to show them to users. What's more. I also tried to involve these values into
## my model and found they have very slight affect to my model's performance. So I just drop them

parsed_df = parsed_df[['description','price']]
parsed_df = parsed_df.dropna()
parsed_df.head()

,description,price
1,"This is ripe and fruity, a wine that is smooth...",15.0
2,"Tart and snappy, the flavors of lime flesh and...",14.0
3,"Pineapple rind, lemon pith and orange blossom ...",13.0
4,"Much like the regular bottling from 2012, this...",65.0
5,Blackberry and raspberry aromas show a typical...,15.0


In [193]:
def priceLevel(row):
    if row['price'] >= 200:
        val = 5
    elif row['price'] >= 100 and row['price'] < 200:
        val = 4
    elif row['price'] >= 70 and row['price'] < 100:
        val = 3
    elif row['price'] >= 35 and row['price'] < 70:
        val = 2
    else:
        val = 1
    return(val)
parsed_df['price_level'] = parsed_df.apply(priceLevel, axis=1)
parsed_df = parsed_df.drop(['price'],1)
parsed_df.head()

,description,price_level
1,"This is ripe and fruity, a wine that is smooth...",1
2,"Tart and snappy, the flavors of lime flesh and...",1
3,"Pineapple rind, lemon pith and orange blossom ...",1
4,"Much like the regular bottling from 2012, this...",2
5,Blackberry and raspberry aromas show a typical...,1


In [194]:
parsed_df = parsed_df.assign(description_length = parsed_df['description'].apply(len))
parsed_df.head()

,description,price_level,description_length
1,"This is ripe and fruity, a wine that is smooth...",1,227
2,"Tart and snappy, the flavors of lime flesh and...",1,186
3,"Pineapple rind, lemon pith and orange blossom ...",1,199
4,"Much like the regular bottling from 2012, this...",2,249
5,Blackberry and raspberry aromas show a typical...,1,261


In [199]:
X = parsed_df['description']
y = parsed_df['price_level']
vectorizer = CountVectorizer()
vectorizer.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [200]:
X = vectorizer.transform(X)
# Training the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

# Testing the model
predictions = rfc.predict(X_test)
print(classification_report(y_test, predictions))

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           1       0.73      0.95      0.82      7736
           2       0.62      0.34      0.44      3342
           3       0.83      0.12      0.20       642
           4       0.92      0.08      0.15       304
           5       1.00      0.03      0.05        73

   micro avg       0.71      0.71      0.71     12097
   macro avg       0.82      0.30      0.33     12097
weighted avg       0.71      0.71      0.66     12097



In [122]:
X = parsed_df['description']
y = parsed_df['price_level']

# Vectorizing model
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [123]:
# Training model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

# Testing model
predictions = rfc.predict(X_test)
print(classification_report(y_test, predictions))

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           1       0.73      0.94      0.82     15496
           2       0.60      0.35      0.44      6654
           3       0.82      0.11      0.20      1287
           4       0.87      0.06      0.12       610
           5       0.78      0.05      0.09       147

   micro avg       0.71      0.71      0.71     24194
   macro avg       0.76      0.30      0.33     24194
weighted avg       0.70      0.71      0.66     24194



In [124]:
## remove outliear
df = init_df.where(init_df['price'] < 1500)


## I drop these columns because some of them looks unrelated to final results(Unnamed: 0, taster_twitter_handle,designation), the others have 
## too many values(more than 1000 or 10000) so it is impossible to show them to users. What's more. I also tried to involve these values into
## my model and found they have very slight affect to my model's performance. So I just drop them
df = df.drop(['Unnamed: 0','description','taster_twitter_handle','title','region_1', 'region_2','designation','variety', 'taster_name'],1)
df = df.dropna()
df.head()

,country,points,price,province,winery
1,Portugal,87.0,15.0,Douro,Quinta dos Avidagos
2,US,87.0,14.0,Oregon,Rainstorm
3,US,87.0,13.0,Michigan,St. Julian
4,US,87.0,65.0,Oregon,Sweet Cheeks
5,Spain,87.0,15.0,Northern Spain,Tandem


In [125]:
# use the LabelEncoder function to convert the country to corresponding integers.
le_country = LabelEncoder()
le_country.fit(df['country'])
df['country']= le_country.transform(df['country'])
# le_country_mapping = dict(zip(le_country.classes_, le_country.transform(le_country.classes_)))
# print(le_country_mapping)
df.head()

,country,points,price,province,winery
1,30,87.0,15.0,Douro,Quinta dos Avidagos
2,39,87.0,14.0,Oregon,Rainstorm
3,39,87.0,13.0,Michigan,St. Julian
4,39,87.0,65.0,Oregon,Sweet Cheeks
5,36,87.0,15.0,Northern Spain,Tandem


In [126]:
# use the LabelEncoder function to convert the province to corresponding integers.
le_province = LabelEncoder()
le_province.fit(df['province'])
df['province']= le_province.transform(df['province'])
# le_province_mapping = dict(zip(le_country.classes_, le_country.transform(le_country.classes_)))
# print(le_name_mapping)
df.head()

,country,points,price,province,winery
1,30,87.0,15.0,107,Quinta dos Avidagos
2,39,87.0,14.0,266,Rainstorm
3,39,87.0,13.0,216,St. Julian
4,39,87.0,65.0,266,Sweet Cheeks
5,36,87.0,15.0,260,Tandem


In [127]:
# use the LabelEncoder function to convert the taster_name to corresponding integers.
le_taster_name = LabelEncoder()
le_taster_name.fit(df['winery'])
df['winery']= le_taster_name.transform(df['winery'])
# le_province_mapping = dict(zip(le_country.classes_, le_country.transform(le_country.classes_)))
# print(le_name_mapping)
df.head()

,country,points,price,province,winery
1,30,87.0,15.0,107,12187
2,39,87.0,14.0,266,12247
3,39,87.0,13.0,216,13563
4,39,87.0,65.0,266,13788
5,36,87.0,15.0,260,13868


In [128]:
def GoodRating(row):
    if row['points'] >= 92:
        val = 3
    elif row['points'] >= 86 and row['points'] < 92:
        val = 2
    else:
        val = 1
    return(val)
df['quality_level'] = df.apply(GoodRating, axis=1)
df = df.drop(['points'],1)
df.head()

,country,price,province,winery,quality_level
1,30,15.0,107,12187,2
2,39,14.0,266,12247,2
3,39,13.0,216,13563,2
4,39,65.0,266,13788,2
5,36,15.0,260,13868,2


In [129]:
X1 = df.drop(["quality_level"], axis=1)
# Extract the target feature
y1 = df['quality_level']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [130]:
def print_score(y_true,y_pred):
    accuracy_dt =  accuracy_score(y_true,y_pred)*100
    print('Accuracy: ', accuracy_dt)
    precision_dt = precision_score(y_true,y_pred,average='weighted')*100
    print('Precision: ', precision_dt)
    recall_dt = recall_score(y_true,y_pred,average='weighted')*100
    print('Recall: ', recall_dt)
    f1_dt = f1_score(y_true,y_pred,average='weighted')*100
    print('f1 Score: ', f1_dt)

In [131]:
clf_dtc = DecisionTreeClassifier()
clf_dtc.fit(X_train1, y_train1)
y_pred_DecisionTreeClassifier= clf_dtc.predict(X_test1)
print_score(y_test1,y_pred_DecisionTreeClassifier)

Accuracy:  67.40550822926143
Precision:  67.15419020339434
Recall:  67.40550822926143
f1 Score:  67.22465132064954


In [132]:
clf4 = GaussianNB()
clf4.fit(X_train1, y_train1)
y_pred_GaussianNB = clf4.predict(X_test1)
print_score(y_test1,y_pred_GaussianNB)

Accuracy:  67.9430981721942
Precision:  56.05827712711331
Recall:  67.9430981721942
f1 Score:  58.51294272970024


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [133]:
clf5 = RandomForestClassifier()
clf5.fit(X_train1, y_train1)
y_pred_rabdom_for_test = clf5.predict(X_test1)
print_score(y_test1,y_pred_rabdom_for_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy:  69.10098420312629
Precision:  68.13201547982527
Recall:  69.10098420312629
f1 Score:  68.48932401746212


In [134]:
knn_3 = KNeighborsClassifier(3, weights='uniform', p=2, metric='euclidean')
knn_3.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_3 = knn_3.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_3)

Accuracy:  67.91828632867421
Precision:  66.69584540147902
Recall:  67.91828632867421
f1 Score:  67.11605837934844


In [135]:
knn_4 = KNeighborsClassifier(4, weights='uniform', p=2, metric='euclidean')
knn_4.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_4 = knn_4.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_4)

Accuracy:  67.49234968158135
Precision:  66.86297108418238
Recall:  67.49234968158135
f1 Score:  66.59608444510643


In [136]:
knn_2 = KNeighborsClassifier(2, weights='uniform', p=2, metric='euclidean')
knn_2.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_2 = knn_2.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_2)

Accuracy:  64.18410387891821
Precision:  66.35039481790882
Recall:  64.18410387891821
f1 Score:  64.0828335742678


In [137]:
df = init_df[['country','province', 'points']]
df = df.dropna()
df.head()

,country,province,points
0,Italy,Sicily & Sardinia,87
1,Portugal,Douro,87
2,US,Oregon,87
3,US,Michigan,87
4,US,Oregon,87


In [138]:
def GoodRating(row):
    if row['points'] >= 92:
        val = 3
    elif row['points'] >= 86 and row['points'] < 92:
        val = 2
    else:
        val = 1
    return(val)
df['quality_level'] = df.apply(GoodRating, axis=1)
df = df.drop(['points'],1)
df.head()

,country,province,quality_level
0,Italy,Sicily & Sardinia,2
1,Portugal,Douro,2
2,US,Oregon,2
3,US,Michigan,2
4,US,Oregon,2


In [139]:
# use the LabelEncoder function to convert the country to corresponding integers.
le_country = LabelEncoder()
le_country.fit(df['country'])
df['country']= le_country.transform(df['country'])
# le_country_mapping = dict(zip(le_country.classes_, le_country.transform(le_country.classes_)))
# print(le_country_mapping)
df.head()

,country,province,quality_level
0,22,Sicily & Sardinia,2
1,31,Douro,2
2,40,Oregon,2
3,40,Michigan,2
4,40,Oregon,2


In [140]:
# use the LabelEncoder function to convert the province to corresponding integers.
le_province = LabelEncoder()
le_province.fit(df['province'])
df['province']= le_province.transform(df['province'])
# le_province_mapping = dict(zip(le_country.classes_, le_country.transform(le_country.classes_)))
# print(le_name_mapping)
df.head()

,country,province,quality_level
0,22,331,2
1,31,108,2
2,40,268,2
3,40,218,2
4,40,268,2


In [141]:
X1 = df.drop(["quality_level"], axis=1)
# Extract the target feature
y1 = df['quality_level']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [142]:
clf_dtc = DecisionTreeClassifier()
clf_dtc.fit(X_train1, y_train1)
y_pred_DecisionTreeClassifier= clf_dtc.predict(X_test1)
print_score(y_test1,y_pred_DecisionTreeClassifier)

Accuracy:  65.76476021861288
Precision:  62.826594333303234
Recall:  65.76476021861288
f1 Score:  54.05614493134853


In [143]:
clf4 = GaussianNB()
clf4.fit(X_train1, y_train1)
y_pred_GaussianNB = clf4.predict(X_test1)
print_score(y_test1,y_pred_GaussianNB)

Accuracy:  65.36833192209993
Precision:  42.73018818277827
Recall:  65.36833192209993
f1 Score:  51.67880414117884


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [144]:
clf5 = RandomForestClassifier()
clf5.fit(X_train1, y_train1)
y_pred_rabdom_for_test = clf5.predict(X_test1)
print_score(y_test1,y_pred_rabdom_for_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy:  65.77245785543838
Precision:  62.79612679914878
Recall:  65.77245785543838
f1 Score:  54.08443904240108


In [145]:
knn_4 = KNeighborsClassifier(4, weights='uniform', p=2, metric='euclidean')
knn_4.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_4 = knn_4.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_4)

Accuracy:  51.035332153029024
Precision:  52.9868294572024
Recall:  51.035332153029024
f1 Score:  49.95971226011501


In [146]:
knn_3 = KNeighborsClassifier(3, weights='uniform', p=2, metric='euclidean')
knn_3.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_3 = knn_3.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_3)

Accuracy:  49.32260795935648
Precision:  53.350609355161225
Recall:  49.32260795935648
f1 Score:  49.18058784329298


In [219]:
df = init_df[['taster_name', 'points']]
df = df.dropna()
df.head()

,taster_name,points
0,Kerin O’Keefe,87
1,Roger Voss,87
2,Paul Gregutt,87
3,Alexander Peartree,87
4,Paul Gregutt,87


In [225]:
df['quality_level'] = df.apply(GoodRating, axis=1)
df = df.drop(['points'],1)
df.head()


,taster_name,quality_level
0,9,2
1,15,2
2,14,2
3,0,2
4,14,2


In [150]:
df['quality_level'] = df.apply(GoodRating, axis=1)
df = df.drop(['points'],1)
df.head()

,country,province,taster_name,price_level,quality_level
1,Portugal,Douro,Roger Voss,1,2
2,US,Oregon,Paul Gregutt,1,2
3,US,Michigan,Alexander Peartree,1,2
4,US,Oregon,Paul Gregutt,2,2
5,Spain,Northern Spain,Michael Schachner,1,2


In [226]:
# use the LabelEncoder function to convert the country to corresponding integers.
# le_country = LabelEncoder()
# le_country.fit(df['country'])
# df['country']= le_country.transform(df['country'])
# # le_country_mapping = dict(zip(le_country.classes_, le_country.transform(le_country.classes_)))
# # print(le_country_mapping)
# # use the LabelEncoder function to convert the country to corresponding integers.
# le_province = LabelEncoder()
# le_province.fit(df['province'])
# df['province']= le_province.transform(df['province'])
le_taster_name = LabelEncoder()
le_taster_name.fit(df['taster_name'])
df['taster_name']= le_taster_name.transform(df['taster_name'])
df.head()

,taster_name,quality_level
0,9,2
1,15,2
2,14,2
3,0,2
4,14,2


In [227]:
X1 = df.drop(["quality_level"], axis=1)
# Extract the target feature
y1 = df['quality_level']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [228]:
clf_dtc = DecisionTreeClassifier()
clf_dtc.fit(X_train1, y_train1)
y_pred_DecisionTreeClassifier= clf_dtc.predict(X_test1)
print_score(y_test1,y_pred_DecisionTreeClassifier)

Accuracy:  67.8444037404801
Precision:  46.02863118901269
Recall:  67.8444037404801
f1 Score:  54.84678686121923


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [229]:
clf5 = RandomForestClassifier()
clf5.fit(X_train1, y_train1)
y_pred_rabdom_for_test = clf5.predict(X_test1)
print_score(y_test1,y_pred_rabdom_for_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy:  67.8444037404801
Precision:  46.02863118901269
Recall:  67.8444037404801
f1 Score:  54.84678686121923


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [230]:
clf4 = GaussianNB()
clf4.fit(X_train1, y_train1)
y_pred_GaussianNB = clf4.predict(X_test1)
print_score(y_test1,y_pred_GaussianNB)

Accuracy:  67.8444037404801
Precision:  46.02863118901269
Recall:  67.8444037404801
f1 Score:  54.84678686121923


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [231]:
knn_3 = KNeighborsClassifier(3, weights='uniform', p=2, metric='euclidean')
knn_3.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_3 = knn_3.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_3)

Accuracy:  51.52800539863106
Precision:  53.33860879603343
Recall:  51.52800539863106
f1 Score:  51.886126375651344


In [232]:
knn_4 = KNeighborsClassifier(4, weights='uniform', p=2, metric='euclidean')
knn_4.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_4 = knn_4.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_4)

Accuracy:  58.459462064976385
Precision:  55.35588280328536
Recall:  58.459462064976385
f1 Score:  56.53273204989427


In [233]:
df = init_df[['winery','points']]
df = df.dropna()
df['quality_level'] = df.apply(GoodRating, axis=1)
df = df.drop(['points'],1)
le_taster_name = LabelEncoder()
le_taster_name.fit(df['winery'])
df['winery']= le_taster_name.transform(df['winery'])
df.head()

,winery,quality_level
0,11608,2
1,12956,2
2,13018,2
3,14390,2
4,14621,2


In [234]:
X1 = df.drop(["quality_level"], axis=1)
# Extract the target feature
y1 = df['quality_level']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [235]:
clf_dtc = DecisionTreeClassifier()
clf_dtc.fit(X_train1, y_train1)
y_pred_DecisionTreeClassifier= clf_dtc.predict(X_test1)
print_score(y_test1,y_pred_DecisionTreeClassifier)

Accuracy:  67.67070590498173
Precision:  65.3829712098542
Recall:  67.67070590498173
f1 Score:  65.2189216562984


In [236]:
clf5 = RandomForestClassifier()
clf5.fit(X_train1, y_train1)
y_pred_rabdom_for_test = clf5.predict(X_test1)
print_score(y_test1,y_pred_rabdom_for_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy:  67.68609347951529
Precision:  65.31927761646453
Recall:  67.68609347951529
f1 Score:  65.28411947964146


In [237]:
clf4 = GaussianNB()
clf4.fit(X_train1, y_train1)
y_pred_GaussianNB = clf4.predict(X_test1)
print_score(y_test1,y_pred_GaussianNB)

Accuracy:  65.45874206578188
Precision:  42.84846912834562
Recall:  65.45874206578188
f1 Score:  51.79353909424772


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [238]:
knn_3 = KNeighborsClassifier(3, weights='uniform', p=2, metric='euclidean')
knn_3.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_3 = knn_3.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_3)

Accuracy:  62.94672052317753
Precision:  61.43534300391968
Recall:  62.94672052317753
f1 Score:  61.96070222655328


In [239]:
knn_4 = KNeighborsClassifier(4, weights='uniform', p=2, metric='euclidean')
knn_4.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_4 = knn_4.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_4)

Accuracy:  63.45451048278515
Precision:  61.84170230481867
Recall:  63.45451048278515
f1 Score:  61.97130287331116


In [161]:
df = init_df[['country', 'province', 'winery', 'price', 'points']]
df = df.dropna()
df.head()

,country,province,winery,price,points
1,Portugal,Douro,Quinta dos Avidagos,15.0,87
2,US,Oregon,Rainstorm,14.0,87
3,US,Michigan,St. Julian,13.0,87
4,US,Oregon,Sweet Cheeks,65.0,87
5,Spain,Northern Spain,Tandem,15.0,87


In [164]:
# use the LabelEncoder function to convert the country to corresponding integers.
le_country = LabelEncoder()
le_country.fit(df['country'])
df['country']= le_country.transform(df['country'])
# le_country_mapping = dict(zip(le_country.classes_, le_country.transform(le_country.classes_)))
# print(le_country_mapping)
# use the LabelEncoder function to convert the country to corresponding integers.
le_province = LabelEncoder()
le_province.fit(df['province'])
df['province']= le_province.transform(df['province'])
le_taster_name = LabelEncoder()
le_taster_name.fit(df['winery'])
df['winery']= le_taster_name.transform(df['winery'])
df.head()

,country,province,winery,price,points
1,30,107,12189,15.0,87
2,39,266,12249,14.0,87
3,39,216,13565,13.0,87
4,39,266,13790,65.0,87
5,36,260,13870,15.0,87


In [166]:
df['price_level'] = df.apply(priceLevel, axis=1)
df = df.drop(['price'],1)
df.head()
df['quality_level'] = df.apply(GoodRating, axis=1)
df = df.drop(['points'],1)
df.head()

,country,province,winery,price_level,quality_level
1,30,107,12189,1,2
2,39,266,12249,1,2
3,39,216,13565,1,2
4,39,266,13790,2,2
5,36,260,13870,1,2


In [167]:
X1 = df.drop(["quality_level"], axis=1)
# Extract the target feature
y1 = df['quality_level']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [168]:
clf_dtc = DecisionTreeClassifier()
clf_dtc.fit(X_train1, y_train1)
y_pred_DecisionTreeClassifier= clf_dtc.predict(X_test1)
print_score(y_test1,y_pred_DecisionTreeClassifier)

Accuracy:  68.91333112801853
Precision:  67.45384821127266
Recall:  68.91333112801853
f1 Score:  67.79789601088915


In [169]:
clf5 = RandomForestClassifier()
clf5.fit(X_train1, y_train1)
y_pred_rabdom_for_test = clf5.predict(X_test1)
print_score(y_test1,y_pred_rabdom_for_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy:  68.80582203109495
Precision:  67.06433543000819
Recall:  68.80582203109495
f1 Score:  67.38475121059041


In [170]:
clf4 = GaussianNB()
clf4.fit(X_train1, y_train1)
y_pred_GaussianNB = clf4.predict(X_test1)
print_score(y_test1,y_pred_GaussianNB)

Accuracy:  67.93747932517367
Precision:  55.43100789799886
Recall:  67.93747932517367
f1 Score:  59.42679059799229


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [171]:
knn_3 = KNeighborsClassifier(3, weights='uniform', p=2, metric='euclidean')
knn_3.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_3 = knn_3.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_3)

Accuracy:  65.80383724776712
Precision:  64.48870125160285
Recall:  65.80383724776712
f1 Score:  64.92349015023937


In [172]:
knn_4 = KNeighborsClassifier(4, weights='uniform', p=2, metric='euclidean')
knn_4.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_4 = knn_4.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_4)

Accuracy:  65.66738339397949
Precision:  65.0352892881046
Recall:  65.66738339397949
f1 Score:  64.63268750845941


In [206]:
df = init_df[['country', 'province', 'winery', 'taster_name', 'price', 'points','description']]
df = df.assign(description_length = parsed_df['description'].apply(len))
df = df.drop(['description'], axis=1)
df = df.dropna()
df.head()

,country,province,winery,taster_name,price,points,description_length
1,Portugal,Douro,Quinta dos Avidagos,Roger Voss,15.0,87,227.0
2,US,Oregon,Rainstorm,Paul Gregutt,14.0,87,186.0
3,US,Michigan,St. Julian,Alexander Peartree,13.0,87,199.0
4,US,Oregon,Sweet Cheeks,Paul Gregutt,65.0,87,249.0
5,Spain,Northern Spain,Tandem,Michael Schachner,15.0,87,261.0


In [207]:
le_country = LabelEncoder()
le_country.fit(df['country'])
df['country']= le_country.transform(df['country'])
# le_country_mapping = dict(zip(le_country.classes_, le_country.transform(le_country.classes_)))
# print(le_country_mapping)
# use the LabelEncoder function to convert the country to corresponding integers.
le_province = LabelEncoder()
le_province.fit(df['province'])
df['province']= le_province.transform(df['province'])
le_winery = LabelEncoder()
le_winery.fit(df['winery'])
df['winery']= le_winery.transform(df['winery'])
le_taster_name = LabelEncoder()
le_taster_name.fit(df['taster_name'])
df['taster_name']= le_taster_name.transform(df['taster_name'])
df['quality_level'] = df.apply(GoodRating, axis=1)
df = df.drop(['points'],1)
df.head()

,country,province,winery,taster_name,price,description_length,quality_level
1,30,105,10743,15,15.0,227.0,2
2,39,263,10793,14,14.0,186.0,2
3,39,213,11893,0,13.0,199.0,2
4,39,263,12064,14,65.0,249.0,2
5,36,257,12132,12,15.0,261.0,2


In [208]:
X1 = df.drop(["quality_level"], axis=1)
# Extract the target feature
y1 = df['quality_level']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [209]:
clf_dtc = DecisionTreeClassifier()
clf_dtc.fit(X_train1, y_train1)
y_pred_DecisionTreeClassifier= clf_dtc.predict(X_test1)
print_score(y_test1,y_pred_DecisionTreeClassifier)

Accuracy:  69.47570398796867
Precision:  69.74521777228867
Recall:  69.47570398796867
f1 Score:  69.59792158138686


In [210]:
clf5 = RandomForestClassifier()
clf5.fit(X_train1, y_train1)
y_pred_rabdom_for_test = clf5.predict(X_test1)
print_score(y_test1,y_pred_rabdom_for_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy:  75.65212881812997
Precision:  74.74666805677855
Recall:  75.65212881812997
f1 Score:  74.8495595191294


In [211]:
clf4 = GaussianNB()
clf4.fit(X_train1, y_train1)
y_pred_GaussianNB = clf4.predict(X_test1)
print_score(y_test1,y_pred_GaussianNB)

Accuracy:  71.716019291604
Precision:  70.16339171684464
Recall:  71.716019291604
f1 Score:  69.24096765356532


In [212]:
knn_3 = KNeighborsClassifier(3, weights='uniform', p=2, metric='euclidean')
knn_3.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_3 = knn_3.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_3)

Accuracy:  71.41523621843075
Precision:  70.17779912802689
Recall:  71.41523621843075
f1 Score:  70.58823029443417


In [213]:
knn_4 = KNeighborsClassifier(4, weights='uniform', p=2, metric='euclidean')
knn_4.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_4 = knn_4.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_4)

Accuracy:  71.26484468184411
Precision:  70.72048287017816
Recall:  71.26484468184411
f1 Score:  70.37393313151932


In [267]:
df = init_df[['country', 'province', 'winery', 'variety', 'taster_name', 'price', 'points','description']]
df = df.assign(description_length = parsed_df['description'].apply(len))
df = df.drop(['description'], axis=1)
df = df.dropna()
le_country = LabelEncoder()
le_country.fit(df['country'])
df['country']= le_country.transform(df['country'])
# le_country_mapping = dict(zip(le_country.classes_, le_country.transform(le_country.classes_)))
# print(le_country_mapping)
# use the LabelEncoder function to convert the country to corresponding integers.
le_province = LabelEncoder()
le_province.fit(df['province'])
df['province']= le_province.transform(df['province'])
le_taster_name = LabelEncoder()
le_taster_name.fit(df['taster_name'])
df['taster_name']= le_taster_name.transform(df['taster_name'])
le_winery = LabelEncoder()
le_winery.fit(df['winery'])
df['winery']= le_winery.transform(df['winery'])
le_variety = LabelEncoder()
le_variety.fit(df['variety'])
df['variety']= le_variety.transform(df['variety'])
df['quality_level'] = df.apply(GoodRating, axis=1)
df = df.drop(['points'],1)
df.head()

,country,province,winery,variety,taster_name,price,description_length,quality_level
1,30,105,10743,415,15,15.0,227.0,2
2,39,263,10793,401,14,14.0,186.0,2
3,39,213,11893,443,0,13.0,199.0,2
4,39,263,12064,405,14,65.0,249.0,2
5,36,257,12132,548,12,15.0,261.0,2


### Model with best result

In [268]:
X1 = df.drop(["quality_level"], axis=1)
# Extract the target feature
y1 = df['quality_level']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [269]:
clf_dtc = DecisionTreeClassifier()
clf_dtc.fit(X_train1, y_train1)
y_pred_DecisionTreeClassifier= clf_dtc.predict(X_test1)
print_score(y_test1,y_pred_DecisionTreeClassifier)

Accuracy:  70.00985323860395
Precision:  70.12798738104576
Recall:  70.00985323860395
f1 Score:  70.06544579514265


In [271]:
clf5 = RandomForestClassifier()
clf5.fit(X_train1, y_train1)
y_pred_rabdom_for_test = clf5.predict(X_test1)
print_score(y_test1,y_pred_rabdom_for_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy:  76.82933153554944
Precision:  76.00906336365354
Recall:  76.82933153554944
f1 Score:  76.06932389402525


In [252]:
df = init_df[['country', 'province', 'winery', 'variety', 'designation', 'taster_name', 'price', 'points','description']]
df = df.assign(description_length = parsed_df['description'].apply(len))
df = df.drop(['description'], axis=1)
df = df.dropna()
le_country = LabelEncoder()
le_country.fit(df['country'])
df['country']= le_country.transform(df['country'])
# le_country_mapping = dict(zip(le_country.classes_, le_country.transform(le_country.classes_)))
# print(le_country_mapping)
# use the LabelEncoder function to convert the country to corresponding integers.
le_province = LabelEncoder()
le_province.fit(df['province'])
df['province']= le_province.transform(df['province'])
le_taster_name = LabelEncoder()
le_taster_name.fit(df['taster_name'])
df['taster_name']= le_taster_name.transform(df['taster_name'])
le_winery = LabelEncoder()
le_winery.fit(df['winery'])
df['winery']= le_winery.transform(df['winery'])
le_variety = LabelEncoder()
le_variety.fit(df['variety'])
df['variety']= le_variety.transform(df['variety'])
le_designation = LabelEncoder()
le_designation.fit(df['designation'])
df['designation']= le_designation.transform(df['designation'])
df['quality_level'] = df.apply(GoodRating, axis=1)
df = df.drop(['points'],1)
df.head()


,country,province,winery,variety,designation,taster_name,price,description_length,quality_level
1,30,102,7873,381,1900,15,15.0,227.0,2
3,38,204,8757,405,22412,0,13.0,199.0,2
4,38,251,8887,372,29079,14,65.0,249.0,2
5,35,245,8941,503,1616,12,15.0,261.0,2
6,21,309,9138,155,2438,9,16.0,181.0,2


In [253]:
X1 = df.drop(["quality_level"], axis=1)
# Extract the target feature
y1 = df['quality_level']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [254]:
clf_dtc = DecisionTreeClassifier()
clf_dtc.fit(X_train1, y_train1)
y_pred_DecisionTreeClassifier= clf_dtc.predict(X_test1)
print_score(y_test1,y_pred_DecisionTreeClassifier)

Accuracy:  69.57951945080092
Precision:  69.6685229420752
Recall:  69.57951945080092
f1 Score:  69.62089483122358


In [255]:
clf5 = RandomForestClassifier()
clf5.fit(X_train1, y_train1)
y_pred_rabdom_for_test = clf5.predict(X_test1)
print_score(y_test1,y_pred_rabdom_for_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy:  76.24427917620137
Precision:  75.47344743319108
Recall:  76.24427917620137
f1 Score:  75.37244665482528


In [248]:
df = init_df[['country', 'province', 'winery', 'variety', 'region_1', 'taster_name', 'price', 'points','description']]
df = df.assign(description_length = parsed_df['description'].apply(len))
df = df.drop(['description'], axis=1)
df = df.dropna()
le_country = LabelEncoder()
le_country.fit(df['country'])
df['country']= le_country.transform(df['country'])
# le_country_mapping = dict(zip(le_country.classes_, le_country.transform(le_country.classes_)))
# print(le_country_mapping)
# use the LabelEncoder function to convert the country to corresponding integers.
le_province = LabelEncoder()
le_province.fit(df['province'])
df['province']= le_province.transform(df['province'])
le_taster_name = LabelEncoder()
le_taster_name.fit(df['taster_name'])
df['taster_name']= le_taster_name.transform(df['taster_name'])
le_winery = LabelEncoder()
le_winery.fit(df['winery'])
df['winery']= le_winery.transform(df['winery'])
le_variety = LabelEncoder()
le_variety.fit(df['variety'])
df['variety']= le_variety.transform(df['variety'])
le_designation = LabelEncoder()
le_designation.fit(df['region_1'])
df['region_1']= le_designation.transform(df['region_1'])
df['quality_level'] = df.apply(GoodRating, axis=1)
df = df.drop(['points'],1)
df.head()



,country,province,winery,variety,region_1,taster_name,price,description_length,quality_level
2,6,43,9307,288,1079,12,14.0,186.0,2
3,6,30,10213,316,491,0,13.0,199.0,2
4,6,43,10337,292,1079,12,65.0,249.0,2
5,5,39,10394,394,663,11,15.0,261.0,2
6,4,49,10601,113,1066,8,16.0,181.0,2


In [249]:
X1 = df.drop(["quality_level"], axis=1)
# Extract the target feature
y1 = df['quality_level']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [250]:
clf_dtc = DecisionTreeClassifier()
clf_dtc.fit(X_train1, y_train1)
y_pred_DecisionTreeClassifier= clf_dtc.predict(X_test1)
print_score(y_test1,y_pred_DecisionTreeClassifier)

Accuracy:  69.0654931400466
Precision:  69.42801553065053
Recall:  69.0654931400466
f1 Score:  69.22820746003609


In [251]:
clf5 = RandomForestClassifier()
clf5.fit(X_train1, y_train1)
y_pred_rabdom_for_test = clf5.predict(X_test1)
print_score(y_test1,y_pred_rabdom_for_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy:  76.42376391405644
Precision:  75.50045639595277
Recall:  76.42376391405644
f1 Score:  75.63659511151684


In [260]:
df = init_df[['country', 'province', 'winery', 'variety','designation', 'region_1', 'taster_name', 'price', 'points','description']]
df = df.assign(description_length = parsed_df['description'].apply(len))
df = df.drop(['description'], axis=1)
df = df.dropna()
le_country = LabelEncoder()
le_country.fit(df['country'])
df['country']= le_country.transform(df['country'])
# le_country_mapping = dict(zip(le_country.classes_, le_country.transform(le_country.classes_)))
# print(le_country_mapping)
# use the LabelEncoder function to convert the country to corresponding integers.
le_province = LabelEncoder()
le_province.fit(df['province'])
df['province']= le_province.transform(df['province'])
le_taster_name = LabelEncoder()
le_taster_name.fit(df['taster_name'])
df['taster_name']= le_taster_name.transform(df['taster_name'])
le_winery = LabelEncoder()
le_winery.fit(df['winery'])
df['winery']= le_winery.transform(df['winery'])
le_variety = LabelEncoder()
le_variety.fit(df['variety'])
df['variety']= le_variety.transform(df['variety'])
le_designation = LabelEncoder()
le_designation.fit(df['designation'])
df['designation']= le_designation.transform(df['designation'])
le_region_1 = LabelEncoder()
le_region_1.fit(df['region_1'])
df['region_1']= le_region_1.transform(df['region_1'])
df['quality_level'] = df.apply(GoodRating, axis=1)
df = df.drop(['points'],1)
df.head()

,country,province,winery,variety,designation,region_1,taster_name,price,description_length,quality_level
3,6,30,7357,298,17825,443,0,13.0,199.0,2
4,6,42,7453,276,23027,992,12,65.0,249.0,2
5,5,38,7498,369,1196,604,11,15.0,261.0,2
6,4,48,7671,105,1868,979,8,16.0,181.0,2
9,3,0,4619,272,12871,17,13,27.0,160.0,2


In [261]:
X1 = df.drop(["quality_level"], axis=1)
# Extract the target feature
y1 = df['quality_level']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [262]:
clf_dtc = DecisionTreeClassifier()
clf_dtc.fit(X_train1, y_train1)
y_pred_DecisionTreeClassifier= clf_dtc.predict(X_test1)
print_score(y_test1,y_pred_DecisionTreeClassifier)

Accuracy:  69.0972863208418
Precision:  69.50577738995057
Recall:  69.0972863208418
f1 Score:  69.2781851648091


In [263]:
clf5 = RandomForestClassifier()
clf5.fit(X_train1, y_train1)
y_pred_rabdom_for_test = clf5.predict(X_test1)
print_score(y_test1,y_pred_rabdom_for_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy:  76.07531844194203
Precision:  75.15990533466935
Recall:  76.07531844194203
f1 Score:  75.20589448904656


In [264]:
clf4 = GaussianNB()
clf4.fit(X_train1, y_train1)
y_pred_GaussianNB = clf4.predict(X_test1)
print_score(y_test1,y_pred_GaussianNB)

Accuracy:  70.6756507291859
Precision:  68.61730529797822
Recall:  70.6756507291859
f1 Score:  67.65550132045975


In [265]:
knn_3 = KNeighborsClassifier(3, weights='uniform', p=2, metric='euclidean')
knn_3.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_3 = knn_3.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_3)

Accuracy:  66.25438434557873
Precision:  64.643147895664
Recall:  66.25438434557873
f1 Score:  65.2256812933911


In [266]:
knn_4 = KNeighborsClassifier(4, weights='uniform', p=2, metric='euclidean')
knn_4.fit(X_train1, y_train1)
y_pred_KNeighborsClassifier_4 = knn_4.predict(X_test1)
print_score(y_test1,y_pred_KNeighborsClassifier_4)

Accuracy:  66.9004984308658
Precision:  64.53163944226094
Recall:  66.9004984308658
f1 Score:  64.54279351470907
